In [1]:
include("../src/NursesScheduling.jl")
using .NurseSchedules
using Logging
using JSON

import Base.in
""

""

In [2]:
Shifts = Array{String,2}
BestResult = @NamedTuple{shifts::Shifts, score::Int}

ITERATION_NUMBER = 50
TABU_MAX_SIZE = 10
SCHEDULE_PATH = "../schedules/schedule_2016_august_medium.json"

"../schedules/schedule_2016_august_medium.json"

In [3]:
function in(shifts::Shifts, tabu_list::Vector{BestResult})
    findfirst(record -> record.shifts == shifts, tabu_list) != nothing
end

in (generic function with 35 methods)

In [4]:
nurse_schedule = Schedule(SCHEDULE_PATH)

schedule_shifts = get_shifts(nurse_schedule)
workers, shifts = schedule_shifts
month_info = get_month_info(nurse_schedule)
workers_info = get_workers_info(nurse_schedule)

"Max neighborhood: $(get_max_nbhd_size(nurse_schedule))"

"Max neighborhood: 417"

In [5]:
penalty = score(schedule_shifts, month_info, workers_info)
best_res = BestResult((shifts = shifts, score = penalty))
best_iter_res = best_res
tabu_list = Vector{BestResult}()
push!(tabu_list, best_res)

"Initial score: $(best_res.score)"

"Initial score: 216"

In [6]:
for i in 1:ITERATION_NUMBER
    nbhd = Neighborhood(best_iter_res.shifts)
    for nbr_shifts in nbhd
        candidate_score = score((workers, nbr_shifts), month_info, workers_info)
        if best_iter_res.score > candidate_score && !(nbr_shifts in tabu_list)
            global best_iter_res = BestResult((nbr_shifts, candidate_score))
        end
        if best_res.score > best_iter_res.score
            println("Penalty reduced in iter '$(i)': '$(best_res.score)' -> '$(best_iter_res.score)'")
            global best_res = best_iter_res
            best_res.score == 0 && break
        end
        push!(tabu_list, best_iter_res)
        if length(tabu_list) > TABU_MAX_SIZE
            popfirst!(tabu_list)
        end
    end
    best_res.score == 0 && break
end

Penalty reduced in iter '1': '216' -> '184'
Penalty reduced in iter '1': '184' -> '174'
Penalty reduced in iter '1': '174' -> '170'
Penalty reduced in iter '1': '170' -> '166'
Penalty reduced in iter '2': '166' -> '156'
Penalty reduced in iter '2': '156' -> '146'
Penalty reduced in iter '2': '146' -> '124'
Penalty reduced in iter '3': '124' -> '120'
Penalty reduced in iter '3': '120' -> '114'
Penalty reduced in iter '3': '114' -> '94'
Penalty reduced in iter '4': '94' -> '74'
Penalty reduced in iter '5': '74' -> '54'
Penalty reduced in iter '6': '54' -> '34'
Penalty reduced in iter '7': '34' -> '30'
Penalty reduced in iter '7': '30' -> '20'
Penalty reduced in iter '7': '20' -> '14'
Penalty reduced in iter '8': '14' -> '0'


In [7]:
penalty_improved, errors = score((workers, best_res.shifts), month_info, workers_info, true)
JSON.print(errors, 4)
"Score improved: '$(penalty)' -> '$(penalty_improved)'"

[]


"Score improved: '216' -> '0'"